In [ ]:
# Importin Libraries

import pandas as pd # Data manipulation
import numpy as np # Matrix calculation
import geopandas as gpd # GIS of Pandas
import seaborn as sb # Parof of matplotlib for Data Viz
import matplotlib.pyplot as plt # data viz

# ETL (Extract, Tranform, Load)

### Data Extraction

In [ ]:
# Defining Directories

directory_main = '/users/ruhidmirzayev/palette/notebooks/cohort 6/'
# Yield
directory_rm_yields = directory_main + 'rm-yields-data.csv'
#GIS
directory_gis= directory_main + 'SK_RM_Shapefiles/RuralMunicipality.shp'

In [ ]:
# Reading Yields
df_rm_yields=pd.read_csv(directory_rm_yields)
# Reading GIS
gdf_rm=gpd.read_file(directory_gis)


### Data Transformation

In [ ]:
df_rm_yields.info()

In [ ]:
df_rm_yields.isna().sum()

In [ ]:
df_rm_yields.describe().T


In [ ]:
df_rm_yields

In [ ]:
gdf_rm.plot() # good for GitHub

In [ ]:
df_major_crops=df_rm_yields[['Year', 'RM', 'Canola', 'Spring Wheat',
       'Durum','Oats', 'Lentils', 'Peas', 'Barley']]

In [ ]:
df_major_crops.describe().T
# Spring wheat 198 - investigate
# Oats 165 to invistigate
# 4 digits Lentils to investigate - 1 bushel = 60 pounds


In [ ]:
# Changing Pounds to bushels
df_major_crops['Lentils']=df_major_crops['Lentils']/60

In [ ]:
df_major_crops.describe().T

In [ ]:
gdf_rm_clean=gdf_rm[['RMNO', 'RMNM', 'geometry']]

In [ ]:
print(gdf_rm_clean.info())
print(df_major_crops.info())

In [ ]:
# Need to change Object to Int
gdf_rm_clean['RMNO']=gdf_rm_clean['RMNO'].astype(int)

# Exploratory Data Analysis

## Missing values

## GIS Analysis

In [ ]:
gdf_rm_clean

In [ ]:
# Changin CRS system to regular lon and lat
gdf_rm_clean=gdf_rm_clean.to_crs(4326)

In [ ]:
df_major_crops

In [ ]:
# Merging Yield data with GIS
gdf_rm_yield=pd.merge(gdf_rm_clean.rename(columns={'RMNO':'RM'}), df_major_crops, on='RM', how='inner')

In [ ]:
# looking at unique RM names for GIS data
gdf_rm_yield['RM'].unique()

In [ ]:
# looking at unique RM names for yield data
df_major_crops['RM'].unique()

In [ ]:
# Plot can be displayed in GitHub
gdf_rm_yield[gdf_rm_yield['Year']==2021]\
    .plot(column='Canola',
             cmap='Greens',
             legend=True)
plt.title('Canola Yield 2021', color='teal', size=16)
plt.xticks(color='white')
plt.yticks(color='white')
plt.show()

In [ ]:
# List of crops to include in plots
crops = ['Spring Wheat', 'Durum', 'Oats', 'Lentils', 'Peas', 'Barley', 'Canola']

# List of years to include in subplots
years = list(range(2004, 2023 + 1))

# Function to plot yield data for a specific crop
def plot_yield_by_year(crop):
    # Set up the figure with 4 rows and 5 columns for the 20 subplots
    fig, axs = plt.subplots(4, 5, figsize=(20, 16))
    fig.suptitle(f'{crop} Yield per Year (2004 - 2023)', color='teal', size=20)
    
    # Flatten the axs array for easy indexing
    axs = axs.flatten()

    # Loop through each year and plot it on its respective subplot
    for i, year in enumerate(years):
        ax = axs[i]
        gdf_rm_yield[gdf_rm_yield['Year'] == year].plot(
            column=crop,
            cmap='RdYlGn',
            legend=False,
            ax=ax,
            edgecolor='black'
        )
        ax.set_title(f'Year: {year}', color='teal', size=12)
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_xlabel('')
        ax.set_ylabel('')
    
    # Adjust the spacing between subplots for readability
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

# Generate plots for each crop
for crop in crops:
    plot_yield_by_year(crop)

In [ ]:
# crops - is a list defined in mapping cell
# >0.2 slight correlation
# >0.4 Moderate corrleation
# > 0.6 High
# > 0.8 Very correlation 

# Pearson Correlation
sb.heatmap(df_major_crops.loc[df_major_crops['Year']>2003][crops].corr(),
           annot=True,
           cmap='RdYlGn')

# Rank correlatation

## Outliers

**Before treating**

In [ ]:
# Filter the DataFrame for the years 2004-2023
filtered_df = df_major_crops[(df_major_crops['Year'] >= 2004) & (df_major_crops['Year'] <= 2023)]

# Define the list of crops
crops = ['Canola', 'Spring Wheat', 'Durum', 'Oats', 'Lentils', 'Peas', 'Barley']

# Set up the figure and axes
fig, axes = plt.subplots(nrows=4, ncols=5, figsize=(20, 16))

# Flatten the axes array for easy iteration
axes = axes.flatten()

# Iterate through the years and create a boxplot for each crop
for i, year in enumerate(range(2004, 2023 + 1)):
    if i < len(axes):
        ax = axes[i]
        year_data = filtered_df[filtered_df['Year'] == year]
        year_data.boxplot(column=crops, ax=ax)
        ax.set_title(f'Year: {year}', size=12, color='teal')
        ax.tick_params(axis='x', rotation=30)  # Rotate x-tick labels

# Hide any unused subplots
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

# Adjust layout
plt.tight_layout()
plt.show()


**After treating**

In [ ]:
# Define the list of crops
crops = ['Canola', 'Spring Wheat', 'Durum', 'Oats', 'Lentils', 'Peas', 'Barley']

# Calculate mean and standard deviation for each crop
means = df_major_crops[crops].mean()
stds = df_major_crops[crops].std()

# Determine the clipping bounds
lower_bounds = means - 3 * stds
upper_bounds = means + 3 * stds

# Clip the data
df_clipped = df_major_crops.copy()
for crop in crops:
    df_clipped[crop] = df_major_crops[crop].clip(lower=lower_bounds[crop], upper=upper_bounds[crop])

In [ ]:
# Define the list of crops
crops = ['Canola', 'Spring Wheat', 'Durum', 'Oats', 'Lentils', 'Peas', 'Barley']

# Set up the figure and axes
fig, axes = plt.subplots(nrows=4, ncols=5, figsize=(20, 16))

# Flatten the axes array for easy iteration
axes = axes.flatten()

# Iterate through the years and create a boxplot for each crop
for i, year in enumerate(range(2004, 2023 + 1)):
    if i < len(axes):
        ax = axes[i]
        year_data = df_clipped[df_clipped['Year'] == year]
        year_data.boxplot(column=crops, ax=ax)
        ax.set_title(f'Year: {year}', size=12, color='teal')
        ax.tick_params(axis='x', rotation=30)  # Rotate x-tick labels

# Hide any unused subplots
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

# Adjust layout
plt.tight_layout()
plt.show()


## Histograms

# Feature Construction and Selection

In [ ]:
# We will filter years from 2000-2023

df_00_23=df_major_crops[df_major_crops['Year']>=2000].drop(columns='decade')

In [ ]:
df_00_23

In [ ]:
pd.merge(
    gdf_rm_clean.rename(columns={'RMNO': 'RM'}),
    df_00_23.groupby('RM').mean(),
    on='RM').plot('Canola', cmap='RdYlGn', legend=True)
plt.title('Historical Average | Canola')
plt.show

In [ ]:
merged_df = pd.merge(
    gdf_rm_clean.rename(columns={'RMNO': 'RM'}),
    df_00_23.groupby('RM').mean(),
    on='RM'
)

# List of crops
crops = ['Canola', 'Spring Wheat', 'Durum', 'Oats', 'Lentils', 'Peas', 'Barley']

# Create a subplot for each crop
fig, axes = plt.subplots(nrows=len(crops), figsize=(10, 5 * len(crops)))

# Plot each crop
for i, crop in enumerate(crops):
    merged_df.plot(column=crop, cmap='RdYlGn', legend=True, ax=axes[i])
    axes[i].set_title(f'Historical Average | {crop}')

plt.tight_layout()
plt.show()


In [ ]:
merged_df = pd.merge(
    gdf_rm_clean.rename(columns={'RMNO': 'RM'}),
    df_00_23.groupby('RM').std(),
    on='RM'
)

# List of crops
crops = ['Canola', 'Spring Wheat', 'Durum', 'Oats', 'Lentils', 'Peas', 'Barley']

# Create a subplot for each crop
fig, axes = plt.subplots(nrows=len(crops), figsize=(10, 5 * len(crops)))

# Plot each crop
for i, crop in enumerate(crops):
    merged_df.plot(column=crop, cmap='RdYlGn_r', legend=True, ax=axes[i])
    axes[i].set_title(f'Historical Average | {crop}')

plt.tight_layout()
plt.show()

In [ ]:
# List of crops
crops = ['Canola', 'Spring Wheat', 'Durum', 'Oats', 'Lentils', 'Peas', 'Barley']

# Group by 'RM' and calculate mean and standard deviation for each crop
df_agg_00_23 = df_00_23.groupby('RM')[crops].agg(['mean', 'std'])

# Flatten the column multi-index
df_agg_00_23.columns = ['_'.join(col).strip() for col in df_agg_00_23.columns.values]

# Reset index to make 'RM' a column again
df_agg_00_23.reset_index(inplace=True)


In [ ]:
# Feature Construction Completed
# Mean and STD features are created
df_agg_00_23

- Unsupervised ML - no train, not test and not validation dataset
- We do not standardise or normalize , min/max scaler, bucketizing, data in this spesific dataset, because mean and std of crops are the same unit.

In [ ]:
# Function to bucketize the year into decades
def bucketize_decade(year):
    decade_start = (year // 10) * 10
    decade_end = decade_start + 9
    return f"{decade_start}-{decade_end}"

In [ ]:
# Create a new column 'decade' using the function
df_major_crops['decade'] = df_major_crops['Year'].apply(bucketize_decade)

In [ ]:
crops = ['Canola', 'Spring Wheat', 'Durum', 'Oats', 'Lentils', 'Peas', 'Barley']
df_major_crops[crops+['decade']].groupby('decade').mean()

# ML Modelling

## Spectral Clustering

### Optimal Clusters recommeded by the scores

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import SpectralClustering
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

# Assuming df_agg_00_23 is already loaded
crops = ['Canola', 'Spring Wheat', 'Durum', 'Oats', 'Lentils', 'Peas', 'Barley']

# Function to prepare data for each crop
def prepare_data_for_crop(df, crop):
    columns = [f'{crop}_mean', f'{crop}_std']
    crop_data = df[columns].dropna().values
    return crop_data

# Standardize the data
def standardize_data(data):
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(data)
    return data_scaled

# Function to perform spectral clustering and choose the optimal number of clusters
def spectral_clustering(data, n_clusters):
    clustering = SpectralClustering(n_clusters=n_clusters, assign_labels="discretize", random_state=0)
    labels = clustering.fit_predict(data)
    return labels

# Function to find the optimal number of clusters
def find_optimal_clusters(data, max_k):
    scores = []
    for k in range(2, max_k+1):
        labels = spectral_clustering(data, k)
        score = silhouette_score(data, labels)
        scores.append(score)
    optimal_k = scores.index(max(scores)) + 2
    return optimal_k, scores

# Iterate over each crop and perform clustering
for crop in crops:
    # Prepare the data for the crop
    crop_data = prepare_data_for_crop(df_agg_00_23, crop)
    
    # Standardize the data
    crop_data_scaled = standardize_data(crop_data)
    
    # Find the optimal number of clusters
    optimal_k, scores = find_optimal_clusters(crop_data_scaled, 10)
    
    # Perform spectral clustering with the optimal number of clusters
    labels = spectral_clustering(crop_data_scaled, optimal_k)
    
    # Add the cluster labels to the original dataframe
    df_agg_00_23[f'{crop}_Cluster'] = np.nan
    df_agg_00_23.loc[~df_agg_00_23[[f'{crop}_mean', f'{crop}_std']].isna().any(axis=1), f'{crop}_Cluster'] = labels
    
    # Print the results
    print(f'Optimal number of clusters for {crop}: {optimal_k}')
    print(f'Silhouette scores for {crop}: {scores}')
    
    # Visualize the silhouette scores
    plt.plot(range(2, 11), scores, marker='o')
    plt.title(f'Silhouette Scores for {crop}')
    plt.xlabel('Number of clusters')
    plt.ylabel('Silhouette Score')
    plt.show()
    
    # Visualize the clustering results
    plt.scatter(df_agg_00_23[f'{crop}_mean'], df_agg_00_23[f'{crop}_std'], c=df_agg_00_23[f'{crop}_Cluster'], cmap='viridis')
    plt.title(f'Spectral Clustering Results for {crop}')
    plt.xlabel(f'{crop}_mean')
    plt.ylabel(f'{crop}_std')
    plt.colorbar(label='Cluster')
    plt.show()

### Customized Clusters by Expert

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import SpectralClustering

# Assuming df_agg_00_23 is already loaded
crops = ['Canola', 'Spring Wheat', 'Durum', 'Oats', 'Lentils', 'Peas', 'Barley']

# Function to prepare data for each crop
def prepare_data_for_crop(df, crop):
    columns = [f'{crop}_mean', f'{crop}_std']
    crop_data = df[columns].dropna().values
    return crop_data, df[columns].dropna().index

# Standardize the data
def standardize_data(data):
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(data)
    return data_scaled

# Perform spectral clustering with a fixed number of clusters
def perform_spectral_clustering(data, n_clusters=5):
    clustering = SpectralClustering(n_clusters=n_clusters, assign_labels="discretize", random_state=0)
    labels = clustering.fit_predict(data)
    return labels

# Iterate over each crop and perform clustering
for crop in crops:
    # Prepare the data for the crop
    crop_data, indices = prepare_data_for_crop(df_agg_00_23, crop)
    
    # Standardize the data
    crop_data_scaled = standardize_data(crop_data)
    
    # Perform spectral clustering with 5 clusters
    labels = perform_spectral_clustering(crop_data_scaled, 5)
    
    # Add the cluster labels to the original dataframe
    df_agg_00_23[f'{crop}_Cluster_Spectral'] = np.nan
    df_agg_00_23.loc[indices, f'{crop}_Cluster_Spectral'] = labels

# Display the dataframe with the new cluster columns
df_agg_00_23

### Vizualizing Clustering Raw Outputs

In [ ]:
pd.merge(gdf_rm_clean.rename(columns={'RMNO':'RM'}),
         df_agg_00_23,
          on='RM' )\
            .explore('Canola_Cluster_Spectral',
                     cmap='RdYlGn',
                     scheme='naturalbreaks',
                     k=5)

### Vizualizing Ranked Clusters

In [ ]:
# Ranking by historical yield
df_agg_00_23.groupby('Canola_Cluster_Spectral').mean()['Canola_mean'].sort_values()

In [ ]:
# Ranking Clusters based on mean
clusters_to_replace_canola= {
    2:0,
    4:1,
    0:2,
    3:3,
    1:4
}



df_agg_00_23['Canola_Cluster_Spectral']=df_agg_00_23['Canola_Cluster_Spectral'].replace(to_replace=clusters_to_replace_canola)

In [ ]:
pd.merge(gdf_rm_clean.rename(columns={'RMNO':'RM'}),
         df_agg_00_23,
          on='RM' )\
            .explore('Canola_Cluster_Spectral',
                     cmap='RdYlGn',
                     scheme='naturalbreaks',
                     k=5)

## K-Means Clustering

### Raw Clusters

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Assuming df_agg_00_23 is already loaded
crops = ['Canola', 'Spring Wheat', 'Durum', 'Oats', 'Lentils', 'Peas', 'Barley']

# Function to prepare data for each crop
def prepare_data_for_crop(df, crop):
    columns = [f'{crop}_mean', f'{crop}_std']
    crop_data = df[columns].dropna().values
    return crop_data, df[columns].dropna().index

# Standardize the data
def standardize_data(data):
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(data)
    return data_scaled

# Function to perform K-Means clustering
def kmeans_clustering(data, n_clusters):
    kmeans = KMeans(n_clusters=n_clusters, random_state=0)
    labels = kmeans.fit_predict(data)
    return labels

# Function to find the optimal number of clusters using the Elbow method
def find_optimal_clusters(data, max_k):
    distortions = []
    for k in range(1, max_k+1):
        kmeans = KMeans(n_clusters=k, random_state=0)
        kmeans.fit(data)
        distortions.append(kmeans.inertia_)
    optimal_k = distortions.index(min(distortions[1:])) + 1
    return optimal_k, distortions

# Iterate over each crop and perform clustering
for crop in crops:
    # Prepare the data for the crop
    crop_data, indices = prepare_data_for_crop(df_agg_00_23, crop)
    
    # Standardize the data
    crop_data_scaled = standardize_data(crop_data)
    
    # Find the optimal number of clusters using the Elbow method
    optimal_k, distortions = find_optimal_clusters(crop_data_scaled, 10)
    
    # Perform K-Means clustering with the optimal number of clusters
    optimal_labels = kmeans_clustering(crop_data_scaled, optimal_k)
    
    # Perform K-Means clustering with 5 clusters
    fixed_labels = kmeans_clustering(crop_data_scaled, 5)
    
    # Add the cluster labels to the original dataframe
    df_agg_00_23[f'{crop}_Optimal_Cluster_KMeans'] = np.nan
    df_agg_00_23[f'{crop}_Fixed_Cluster_KMeans'] = np.nan
    df_agg_00_23.loc[indices, f'{crop}_Optimal_Cluster_KMeans'] = optimal_labels
    df_agg_00_23.loc[indices, f'{crop}_Fixed_Cluster_KMeans'] = fixed_labels
    
    # Plot the Elbow method graph
    plt.plot(range(1, 11), distortions, marker='o')
    plt.title(f'Elbow Method for {crop}')
    plt.xlabel('Number of clusters')
    plt.ylabel('Distortion')
    plt.show()

# Display the dataframe with the new cluster columns
df_agg_00_23

In [ ]:
pd.merge(gdf_rm_clean.rename(columns={'RMNO':'RM'}),
         df_agg_00_23,
          on='RM' )\
            .explore('Canola_Fixed_Cluster_KMeans',
                     cmap='RdYlGn',
                     scheme='naturalbreaks',
                     k=5)

### Ranked KMeans Clusters

In [ ]:
# Ranking by historical yield
df_agg_00_23.groupby('Canola_Fixed_Cluster_KMeans').mean()['Canola_mean'].sort_values()

In [ ]:
# Ranking Clusters based on mean
clusters_to_replace_canola_kmeans= {
    4:0,
    2:1,
    1:2,
    0:3,
    3:4
}

df_agg_00_23['Canola_Fixed_Cluster_KMeans']=df_agg_00_23['Canola_Fixed_Cluster_KMeans']\
    .replace(to_replace=clusters_to_replace_canola_kmeans)

In [ ]:
pd.merge(gdf_rm_clean.rename(columns={'RMNO':'RM'}),
         df_agg_00_23,
          on='RM' )\
            .explore('Canola_Fixed_Cluster_KMeans',
                     cmap='RdYlGn',
                     scheme='naturalbreaks',
                     k=5)